<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Лемматизация-и-очистка-текста" data-toc-modified-id="Лемматизация-и-очистка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лемматизация и очистка текста</a></span></li><li><span><a href="#Стоп-слова-и-разделение-на-выборки" data-toc-modified-id="Стоп-слова-и-разделение-на-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Стоп-слова и разделение на выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Decision-Tree-Classifier" data-toc-modified-id="Decision-Tree-Classifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Decision Tree Classifier</a></span></li><li><span><a href="#Random-Forest-Classifier" data-toc-modified-id="Random-Forest-Classifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Random Forest Classifier</a></span></li><li><span><a href="#Cat-Boost-Classifier-(умирает-ядро-на-запуске)" data-toc-modified-id="Cat-Boost-Classifier-(умирает-ядро-на-запуске)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Cat Boost Classifier (умирает ядро на запуске)</a></span></li></ul></li><li><span><a href="#Проверяем-на-тестовой-выборке" data-toc-modified-id="Проверяем-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверяем на тестовой выборке</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Модерация комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

**План по выполнению проекта**

1. Загрузим и подготовьте данные.
2. Обучим разные модели. 
3. Сделаем выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Знакомство с данными

In [1]:
import pandas as pd

import re
import nltk
from nltk.stem import WordNetLemmatizer 
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', dtype = str)
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df = df.drop('Unnamed: 0', axis = 1)
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


### Лемматизация и очистка текста

In [4]:
def lemmatize(text):
    nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
    doc = nlp(text)
    return ''.join([token.lemma_ for token in doc])

def clear_text(text):
    txt = re.sub(r'[^a-zA-Z ]', ' ', text)
    txt = txt.lower()
    return ' '.join(txt.split())

In [5]:
df['text'] = df['text'].apply(lambda x: clear_text(x))
corpus = df['text'].values.astype('U')
corpus[0] = lemmatize(clear_text(corpus[0]))

### Стоп-слова и разделение на выборки

In [6]:
features = df['text']
target = df['toxic'].values

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
                                                           features, target, test_size = 0.1, shuffle = False)

In [8]:
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((143362,), (143362,), (15930,), (15930,))

In [9]:
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
tf_idf = TfidfVectorizer(stop_words = stop_words, lowercase = False)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
features_train_tfidf = tf_idf.fit_transform(features_train)

In [11]:
features_test_tfidf = tf_idf.transform(features_test)

## Обучение

### Logistic Regression

In [23]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

f1_scorer = make_scorer(f1_score, pos_label="0")

In [24]:
params = {
          'max_iter' : range(100,500,100),
          'solver' : ['newton-cg', 'liblinear'],
          'warm_start' : [True, False]
         }

random_log = RandomizedSearchCV(LogisticRegression(), params, n_jobs = -1, cv = 5, scoring = f1_scorer)

random_log.fit(features_train_tfidf, target_train)
print(random_log.best_score_, random_log.best_params_)

0.9746645656455906 {'warm_start': True, 'solver': 'newton-cg', 'max_iter': 400}


### Decision Tree Classifier

In [15]:
params = {
          "min_samples_split": [2, 5, 10],
          "max_depth": [2, 4, 10],
          "min_samples_leaf": [2, 4, 6],
              }

random_dt = RandomizedSearchCV(DecisionTreeClassifier(), params, cv = 5, n_jobs = 1, scoring = f1_scorer)
random_dt.fit(features_train_tfidf, target_train)

print(random_dt.best_params_, random_dt.best_score_)

{'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10} 0.966161344849642


### Random Forest Classifier

In [16]:
params = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' : ['gini', 'entropy']
        }

random_rfc = RandomizedSearchCV(RandomForestClassifier(), params, cv = 5, n_jobs = -1, scoring = f1_scorer)
random_rfc.fit(features_train_tfidf, target_train)

print(random_rfc.best_score_, random_rfc.best_params_)

0.9465376800939316 {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 8, 'criterion': 'entropy'}


## Проверяем на тестовой выборке

In [25]:
f1_score(random_log.predict(features_test_tfidf), target_test, pos_label = '0')

0.9755060034305317

## Выводы
1. Открыли и изучили данные
2. Лемматизировали и очистили текст
3. Убрали стоп-слова
4. Разделили на тренировочную и тестовую выборки в соотношении 9:1
5. Обучили следующие модели: **LogisticRegression, DecisionTreeClassifier, RandomForestClassifier**
6. Лучшая метрика на тренировочной выбокре **F1** у модели Логистической Регрессии: 0,974
7. На тестовой выборке метрика **F1** у Логистической Регрессии равна 0,9755